In [347]:
import numpy as np
import datetime
from datetime import timedelta
import pandas as pd
import math
import itertools
import time

In [331]:
g = 4     #gap between consecutive prime numbers
m = 130   #start of search (inclusive)
n = 200   #end of search (inclusive)

In [334]:
#LINHA PARA CRIAR LISTA COM TODOS OS NRS PRIMOS ATE n

primes = []
for possibleprime in range(m, n+1):
    isPrime = True
    for num in range(2, possibleprime):
        if possibleprime % num == 0:
            isPrime = False
    if isPrime:
        primes.append(possibleprime)

In [335]:
primes

[131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]

In [341]:
res = []
for i in range(len(primes)-1):
    if primes[i+1] - primes[i] == g:
        res.append([[primes[i], primes[i+1]]])
    

In [342]:
res

[[[163, 167]], [[193, 197]]]

In [361]:
def gap(g,m,n):
    
    start = time.time()
    
    primes = []
    for possibleprime in range(m, n+1):
        isPrime = True
        for num in range(2, possibleprime):
            if possibleprime % num == 0:
                isPrime = False
        if isPrime:
            primes.append(possibleprime)
    
    resultado = []
    for i in range(len(primes)-1):
        if primes[i+1] - primes[i] == g:
            resultado.append(primes[i])
            resultado.append(primes[i+1])
            break
    
    end = time.time()
            
    print(f'It took {(end-start)*1000}ms to run this shit')
            
    if len(resultado) == 0:
        return None
    else:
        return resultado
    

In [362]:
gap(4,130,200)

It took 0.2789497375488281ms to run this shit


[163, 167]

In [385]:
def gap(g,m,n):
    
    start = time.time()
    
    if m % 2 == 0:
        m = m+1
    
    primes = []
    for possibleprime in range(m, n+1, 2):
        isPrime = True
        for num in range(2, possibleprime):
            if possibleprime % num == 0:
                isPrime = False
        if isPrime:
            primes.append(possibleprime)
    
    
    resultado = [[primes[i], primes[i+1]] for i in range(len(primes)-1) if primes[i+1]-primes[i] == g]
    
    end = time.time()
            
    print(f'It took {(end-start)*1000}ms to run this shit')
            
    if len(resultado) == 0:
        return None
    else:
        return resultado
    

In [358]:
gap(4,130,200)

It took 0.9925365447998047ms to run this shit


[[163, 167], [193, 197]]

In [ ]:
import math

In [390]:
def gap3(g,m,n):
    
    start = time.time()
    
    if m % 2 == 0:
        m = m+1
    
    primes = []
    for num in range(m,n+1,2):
        if all(num%i!=0 for i in range(2,int(math.sqrt(num))+1)):
            primes.append(num)
    
    resultado = []
    for i in range(len(primes)-1):
        if primes[i+1] - primes[i] == g:
            resultado.append(primes[i])
            resultado.append(primes[i+1])
            break
    
    end = time.time()
            
    print(f'It took {(end-start)*1000}ms to run this shit')
            
    if len(resultado) == 0:
        return None
    else:
        return resultado

In [391]:
gap3(4,130,200)

It took 0.031948089599609375ms to run this shit


[163, 167]

In [383]:
primes = []
for num in range(m,n+1):
    if all(num%i!=0 for i in range(2,int(math.sqrt(num))+1)):
        primes.append(num)


In [384]:
primes

[131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]

In [393]:
def gap4(g,m,n):
    
    start = time.time()
    
    if m % 2 == 0:
        m = m+1
    
    primes = []
    for num in range(m,n+1,2):
        if all(num%i!=0 for i in range(2,int(math.sqrt(num))+1)):
            primes.append(num)
            
            
    resultado = [[primes[i], primes[i+1]] for i in range(len(primes)-1) if primes[i+1]-primes[i] == g]
    resultado = resultado[0]
    

    end = time.time()
            
    print(f'It took {(end-start)*1000}ms to run this shit')
            
    if len(resultado) == 0:
        return None
    else:
        return resultado

In [394]:
gap3(4,130,200)

It took 0.03504753112792969ms to run this shit


[163, 167]